In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from imdb import IMDb
from nltk import pos_tag
import rapidfuzz

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\25851\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import nltk
nltk.download("popular")
nltk.download("maxent_ne_chunker")
nltk.download("words")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\25851\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\25851\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\25851\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\25851\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\25851\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

True

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [4]:
links = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
hashtag = re.compile(r'#[\s]+\w+')
tag = re.compile(r'@[^:]+:')
punc = re.compile(r'[^\w\d\s]+')

stop_words = {'.org', 'aahh', 'aarrgghh', 'abt', 'ftl', 'ftw', 'fu', 'fuck', 'fucks', 'gtfo', 'gtg', 'haa',
                          'hah', 'hahah', 'haha', 'hahaha', 'hahahaha', 'hehe', 'heh', 'hehehe', 'hi', 'hihi', 'hihihi',
                          'http', 'https', 'huge', 'huh', 'huhu', 'huhuhu', 'idk', 'iirc', 'im', 'imho', 'imo', 'ini',
                          'irl', 'ish', 'isn', 'isnt', 'j/k', 'jk', 'jus', 'just', 'justwit', 'juz', 'kinda', 'kthx',
                          'kthxbai', 'kyou', 'laa', 'laaa', 'lah', 'lanuch', 'leavg', 'leh', 'lol', 'lols', 'ltd',
                          'mph', 'mrt', 'msg', 'msgs', 'muahahahahaha', 'nb', 'neways', 'ni', 'nice', 'pls', 'plz',
                          'plzz', 'psd', 'pte', 'pwm', 'pwned', 'qfmft', 'qft', 'tis', 'tm', 'tmr', 'tyty', 'tyvm',
                          'um', 'umm', 'viv', 'vn', 'vote', 'voted', 'w00t', 'wa', 'wadever', 'wah', 'wasn', 'wasnt',
                          'wassup', 'wat', 'watcha', 'wateva', 'watever', 'watnot', 'wats', 'wayy', 'wb', 'weren',
                          'werent', 'whaha', 'wham', 'whammy', 'whaow', 'whatcha', 'whatev', 'whateva', 'whatevar',
                          'whatever', 'whatnot', 'whats', 'whatsoever', 'whatz', 'whee', 'whenz', 'whey', 'whore',
                          'whores', 'whoring', 'wo', 'woah', 'woh', 'wooohooo', 'woot', 'wow', 'wrt', 'wtb', 'wtf',
                          'wth', 'wts', 'wtt', 'www', 'xs', 'ya', 'yaah', 'yah', 'yahh', 'yahoocurrency', 'yall', 'yar',
                          'yay', 'yea', 'yeah', 'yeahh', 'yeh', 'yhoo', 'ymmv', 'young', 'youre', 'yr', 'yum', 'yummy',
                          'yumyum', 'yw', 'zomg', 'zz', 'zzz', 'loz', 'lor', 'loh', 'tsk', 'meh', 'lmao', 'wanna',
                          'doesn', 'liao', 'didn', 'didnt', 'omg', 'ohh', 'ohgod', 'hoh', 'hoo', 'bye', 'byee', 'byeee',
                          'byeeee', 'lmaolmao', 'yeahhh', 'yeahhhh', 'yeahhhhh', 'yup', 'yupp', 'hahahahahahaha',
                          'hahahahahah', 'hahhaha', 'wooohoooo', 'wahaha', 'haah', '2moro', 'veh', 'noo', 'nooo',
                          'noooo', 'hahas', 'ooooo', 'ahahaha', 'ahahahahah', 'tomolow', 'accent', 'accented',
                          'accents', 'acne', 'ads', 'afaik', 'aft', 'ago', 'ahead', 'ain', 'aint', 'aircon', 'alot',
                          'am', 'annoy', 'annoyed', 'annoys', 'anycase', 'anymore', 'app', 'apparently', 'apps', 'argh',
                          'ass', 'asses', 'awesome', 'babeh', 'bad', 'bai', 'based', 'bcos', 'bcoz', 'bday', 'bit',
                          'biz', 'blah', 'bleh', 'bless', 'blessed', 'blk', 'blogcatalog', 'bro', 'bros', 'btw', 'byee',
                          'com', 'congrats', 'contd', 'conv', 'cos', 'cost', 'costs', 'couldn', 'couldnt', 'cove',
                          'coves', 'coz', 'crap', 'cum', 'curnews', 'curr', 'cuz', 'dat', 'de', 'diff', 'dis', 'doc',
                          'doesn', 'doesnt', 'don', 'AAWWW', 'dont', 'dr', 'dreamt', 'drs', 'due', 'dun', 'dunno',
                          'duper', 'eh', 'ehh', 'emo', 'emos', 'eng', 'esp', 'fadein', 'ffs', 'fml', 'frm', 'fwah',
                          'g2g', 'gajshost', 'gd', 'geez', 'gg', 'gigs', 'gtfo.1', 'gtg.1', 'hasn', 'hasnt', 'hav',
                          'haven', 'havent', 'hee', 'hello', 'hey', 'hmm', 'ho', 'hohoho', 'lotsa', 'lotta', 'luv',
                          'ly', 'macdailynews', 'nite', 'nom', 'noscript', 'nvr', 'nw', 'ohayo', 'omfg', 'omfgwtf',
                          'omgwtfbbq', 'omw', 'org', 'pf', 'pic', 'pm', 'pmsing', 'ppl', 'pre', 'pro', 'rawr', 'rawrr',
                          'rofl', 'roflmao', 'rss', 'rt', 'sec', 'secs', 'seem', 'seemed', 'seems', 'sgreinfo', 'shd',
                          'shit', 'shits', 'shitz', 'shld', 'shouldn', 'shouldnt', 'shudder', 'sq', 'sqft', 'sqm',
                          'srsly', 'stfu', 'stks', 'su', 'suck', 'sucked', 'sucks', 'suckz', 'sux', 'swf', 'tart',
                          'tat', 'tgif', 'thanky', 'thk', 'thks', 'tht', 'tired', 'hahahahahahahahaha', 'hahahahaha',
                          'hahahahah', 'zzzzz', 'hahahahha', 'lolololol', 'lololol', 'lolol', 'lol', 'dude', 'hmmm',
                          'humm', 'tumblr', 'kkkk', 'fk', 'yayyyyyy', 'fffffffuuuuuuuuuuuu', 'zzzz', 'noooooooooo',
                          'hahahhaha', 'woohoo', 'lalalalalalala', 'lala', 'lalala', 'lalalala', 'whahahaahahahahahah',
                          'hahahahahahahahahahaha', 'ahhh', 'RT', 'rt', 'gif', 'amp', '.com', '.ly', '.net',
                          """'""", ',', '.', ';'}
# stop_words.update(addition_stop)

In [5]:
# Reading JSON data from a file
input_file = 'gg2013.json'
output_file = 'gg2013.csv'

# Converting JSON data to a pandas DataFrame
df = pd.read_json(input_file)

# # Writing DataFrame to a CSV file
# df.to_csv(output_file, index=False)

In [6]:
def remove_stopwords(text):
    # Tokenize the text
    word_tokens = word_tokenize(text)
    filtered_words = []
    for w in word_tokens:
        if w == '&':
            filtered_words.append('and')
        elif w.lower() == 'tv':
            filtered_words.append('telvision')
        elif w.lower() == 'mini-series' or w.lower() == 'miniseries':
            filtered_words.append('mini series')
        else:
            if w.lower() not in stop_words:
                filtered_words.append(w)
    return ' '.join(filtered_words)

# Apply the 'remove_stopwords' function to the 'text' column
df['text'] = df['text'].apply(remove_stopwords)

# Save the DataFrame back to a CSV file
df.to_csv(output_file, index=False)

In [7]:
def tweet_preprocess(tweet):
    tweet = emoji.replace_emoji(tweet, replace='')
    tweet = re.sub(hashtag, '', tweet)
    tweet = re.sub(tag, '', tweet)
    tweet = re.sub(links, '', tweet)
    tweet = re.sub(punc, ' ', tweet)
    return tweet

In [8]:
OFFICIAL_AWARDS = ['cecil b. demille award', 'best motion picture - drama',
                        'best performance by an actress in a motion picture - drama',
                        'best performance by an actor in a motion picture - drama',
                        'best motion picture - comedy or musical',
                        'best performance by an actress in a motion picture - comedy or musical',
                        'best performance by an actor in a motion picture - comedy or musical',
                        'best animated feature film', 'best foreign language film',
                        'best performance by an actress in a supporting role in a motion picture',
                        'best performance by an actor in a supporting role in a motion picture',
                        'best director - motion picture', 'best screenplay - motion picture',
                        'best original score - motion picture', 'best original song - motion picture',
                        'best television series - drama',
                        'best performance by an actress in a television series - drama',
                        'best performance by an actor in a television series - drama',
                        'best television series - comedy or musical',
                        'best performance by an actress in a television series - comedy or musical',
                        'best performance by an actor in a television series - comedy or musical',
                        'best mini series or motion picture made for television',
                        'best performance by an actress in a mini series or motion picture made for television',
                        'best performance by an actor in a mini series or motion picture made for television',
                        'best performance by an actress in a supporting role in a series mini series or motion picture made for television',
                        'best performance by an actor in a supporting role in a series mini series or motion picture made for television']

TITLE = r"(?:[A-Z][a-z]*\.\s*)?"
NAME1 = r"[A-Z][a-z]+,?\s+"
MIDDLE_I = r"(?:[A-Z][a-z]*\.?\s*)?"
NAME2 = r"[A-Z][a-z]+"

WHITE_SPACE = r'\s+'
SINGLE_CHAR = r'(?<!\S)\S(?!\S)'

In [9]:
df['processed'] = df['text'].apply(tweet_preprocess)

In [10]:
df.processed

0                                          JLo  s dress    
1         What  s making Sofia Vergara  s boobs stay lik...
2          Kerry Washington is EVERYTHING Dying over her...
3                          Anne Hathaway has got me living 
4                Jennifer Lopez  s lace dress   Thoughts   
                                ...                        
174638     I was sad that Mandy Patinkin did n t win  he...
174639     Jennifer Lawrence aceptando premio      t co ...
174640    Golden Globes lots of fashion messes   but gla...
174641    Did they have mug shots at the golden globes  ...
174642    Says    I also did n t get the Acting Nominati...
Name: processed, Length: 174643, dtype: object

In [11]:
def altAwardName(OFFICIAL_AWARDS, extra_award, extra_name):
  flag = 0

  for official in OFFICIAL_AWARDS:
    for award in extra_award[official]:
      if "score" in award:
        extra = "score"
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])
      if "song" in award:
        extra = "song"
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])
      if "television" in award:
        extra = award.replace("television", 'tv')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("television", 't.v.')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

      if "motion picture" in award:
        extra = award.replace("motion picture", "movie")
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("motion picture", "film")
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

      if "film" in award:
        extra = award.replace("film", "motion picture")
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("film", "movie")
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

      if "comedy or musical" in award:
        extra = award.replace("comedy or musical", 'comedy')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("comedy or musical", 'musical')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

      if "made for television" in award:
        extra = award.replace("made for television", 'television')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("made for television", 'tv')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("made for television", 't.v.')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

      if "series, mini-series or motion picture made for television" in award:
        extra = award.replace(
            "series, mini-series or motion picture made for television",
            'series')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "series, mini-series or motion picture made for television",
            'mini-series')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "series, mini-series or motion picture made for television",
            'miniseries')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "series, mini-series or motion picture made for television", 'tv')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "series, mini-series or motion picture made for television",
            'television')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "series, mini-series or motion picture made for television",
            'tv movie')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "series, mini-series or motion picture made for television",
            'tv series')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "series, mini-series or motion picture made for television",
            'television series')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

      if "mini-series or motion picture made for television" in award:
        extra = award.replace(
            "mini-series or motion picture made for television", 'miniseries')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "mini-series or motion picture made for television", 'mini-series')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "mini-series or motion picture made for television", 'tv movie')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace(
            "mini-series or motion picture made for television",
            'television movie')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

      if "television series" in award:
        extra = award.replace("television series", 'series')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("television series", 'tv')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("television series", 't.v.')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

        extra = award.replace("television series", 'television')
        if extra not in extra_award[official]:
          flag = 1
          extra_award[official].append(extra)
          extra_name[official].append(
              [item for item in extra.split() if not item in TO_DELETE])

      if "television series - comedy or musical" in award:

        for word in [
            "tv comedy", "tv musical", "comedy series", "t.v. comedy",
            "t.v. musical", "television comedy", "television musical"
        ]:
          extra = award.replace("television series - comedy or musical", word)
          if extra not in extra_award[official]:
            flag = 1
            extra_award[official].append(extra)
            extra_name[official].append(
                [item for item in extra.split() if not item in TO_DELETE])

      if "television series - drama" in award:
        for word in [
            "tv drama", "drama series", "television drama", "t.v. drama"
        ]:
          extra = award.replace("television series - drama", word)
          if extra not in extra_award[official]:
            flag = 1
            extra_award[official].append(extra)
            extra_name[official].append(
                [item for item in extra.split() if not item in TO_DELETE])
  if flag == 1:
    altAwardName(OFFICIAL_AWARDS, extra_award, extra_name)

def getTweetByAward(OFFICIAL_AWARDS, extra_name, tweets):
    tweet_by_award_dict = dict()
    for award in OFFICIAL_AWARDS:
        tweet_by_award_dict[award] = []
    
    OFFICIAL_AWARDS.sort(key=lambda s: len(s), reverse=True)
    for award in OFFICIAL_AWARDS:
        tweet_length = len(tweets)
        for i in range(tweet_length - 1, -1, -1):
            tweet = tweets[i]
            for extra in extra_name[award]:
                if 'actor' in tweet.lower() or 'actress' in tweet.lower():
                    if 'actor' in award or 'actress' in award:
                        if 'actor' in extra or 'actress' in extra:
                            if 'supporting' in tweet.lower():
                                if 'supporting' in award:
                                    if 'supporting' in extra:
                                        flag = True
                                        for word in extra:
                                            if flag == True:
                                                flag = flag and word.lower() in tweet.lower()
    
                                        if flag == True:
                                            tweet_by_award_dict[award].append(tweet)
                                            break
                            else:
                                flag = True
                                for word in extra:
                                    if flag == True:
                                        flag = flag and word.lower() in tweet.lower()
    
                                if flag == True:
                                    tweet_by_award_dict[award].append(tweet)
                                    break
                else:
                    flag = True
                    for word in extra:
                        if flag == True:
                            flag = flag and word.lower() in tweet.lower()
    
                    if flag == True:
                        tweet_by_award_dict[award].append(tweet)
                        break
    return tweet_by_award_dict

In [12]:
tweets = df.processed.to_list()
extra_award = dict()
fresh_names = dict()

In [13]:
from global_var import *

for award in OFFICIAL_AWARDS:
    fresh_names[award] = [[item for item in award.split() if not item in TO_DELETE]]
    extra_award[award] = []
    extra_award[award].append(award)

In [14]:
altAwardName(OFFICIAL_AWARDS, extra_award, fresh_names)
    
tweet_by_award_dict = getTweetByAward(OFFICIAL_AWARDS, fresh_names, tweets)

In [15]:
for key, value in tweet_by_award_dict.items():
    value = list(set(value))
    for i in range(len(value)):
        value[i] = re.sub(WHITE_SPACE, ' ', value[i])
        value[i] = re.sub(SINGLE_CHAR, ' ', value[i])
    value = list(set(value))
    tweet_by_award_dict[key] = value
    

In [16]:
fresh_names

{'cecil b. demille award': [['cecil', 'b.', 'demille']],
 'best motion picture - drama': [['motion', 'picture', 'drama'],
  ['movie', 'drama'],
  ['film', 'drama']],
 'best performance by an actress in a motion picture - drama': [['actress',
   'motion',
   'picture',
   'drama'],
  ['actress', 'movie', 'drama'],
  ['actress', 'film', 'drama']],
 'best performance by an actor in a motion picture - drama': [['actor',
   'motion',
   'picture',
   'drama'],
  ['actor', 'movie', 'drama'],
  ['actor', 'film', 'drama']],
 'best motion picture - comedy or musical': [['motion',
   'picture',
   'comedy',
   'musical'],
  ['movie', 'comedy', 'musical'],
  ['film', 'comedy', 'musical'],
  ['motion', 'picture', 'comedy'],
  ['motion', 'picture', 'musical'],
  ['movie', 'comedy'],
  ['movie', 'musical'],
  ['film', 'comedy'],
  ['film', 'musical']],
 'best performance by an actress in a motion picture - comedy or musical': [['actress',
   'motion',
   'picture',
   'comedy',
   'musical'],
  ['ac

In [17]:
nominees = {}
nominee_names = {}
award_names_extracted = {}
award_stoplist = ['best','award','for','or','made', 'the', 'in', 'a', 'by', 'performance', 'an','golden','globes','role', '-', 'of', 'go', 
                  'b.', 'game', 'yes', 'should', 'could', 'would', 'have', 'been', 'i', 'this', 'that', 'even', 'with', 'I']
name_pattern = re.compile(r'[A-Z][a-z]+\s[A-Z][a-z]+')
person_award = ['actor', 'actress', 'director', 'demille']
tv_related_award = ['tv', 't.v.', 'TV', 'television', 'film', 'motion', 'picture', 'series', 'animated', 'animation']
nomination_words = ["nominated", "nominee", "nominate", "ceremony", "category", "announced", "congratulations", "congrats", "honored", 
                    "prestigious", "deserving", "celebrate", "recognized", "for", "deserves", "recognize", "believe", "overlook", "deserved", "up", "rooting", "hope", "hoping",
                   "finally", "happy for", "robbed", "overrated", "pick", "tough", "competition","earned", "well", "favorite", "clear", "clearly", "well-deserved", "right", "direction"]
verbs = ['was', 'were', 'is', 'are']

In [18]:
for key, value in tweet_by_award_dict.items():
    i = len(value) - 1
    while i >= 0:
        if any(nom_word in value[i] for nom_word in nomination_words):
            words = value[i].split()
            value[i] = ' '.join(['' if word.lower() in award_stoplist else word for word in words])
        else:
            value.pop(i)
        i -= 1

In [19]:
for award in OFFICIAL_AWARDS:
    nominees[award]=[]
    nominee_names[award]=[]
    #award_names_extracted[award] = [a for a in award.split() if a not in award_stoplist]

In [20]:
def extract_name(quote, language='english'):
    tokens = word_tokenize(quote)
    tags = nltk.pos_tag(tokens)
    tree = nltk.ne_chunk(tags)
    names=[]
    for subtree in tree.subtrees():
        if subtree.label() == 'PERSON':
            leave = " ".join(l[0] for l in subtree.leaves())
            names.append(leave)
    return names

In [21]:
# # This dictionary maps each award with lists of tweets that are relevant to the award
# map_award_tweet = {}
# for award in OFFICIAL_AWARDS:
#     map_award_tweet[award] = []
#     for tweet in df['processed']:
#         count = 0
#         for word in award_names_extracted[award]:
#             if word in tweet and any(nom_word in tweet for nom_word in nomination_words):
#                 tweet = tweet.split()
#                 tweet = ' '.join(['' if word.lower() in award_stoplist else word for word in tweet])
#                 map_award_tweet[award].append(tweet)
#                 count += 1
#             if count == 2:
#                 break

In [22]:
# for key, value in tweet_by_award_dict.items():
#     if any([kw in key for kw in person_award]):
#         for tweet in value:
#             names = re.findall(name_pattern, tweet)
#             for name in names:
#                 flag = True
#                 for w in tweet_by_award_dict[key]:
#                     flag = flag and w not in name.lower().split()
#                 if flag:
#                     nominee_names[key].append(name)
#     else:
#         for tweet in value:
#             if any([kw in tweet for kw in tv_related_award]):
#                 if tweet not in nominee_names[key]:
#                     nominee_names[key].append(tweet)

In [25]:
# nominee = {}
# ia = IMDb()
# for key, value in tweet_by_award_dict.items():
#     nominee[key] = {}
#     if any([kw in key for kw in person_award]):
#         for tweet in value:
#             tweet = tweet.split()
#             print(key)
#             tweet = [word for word in tweet if word.lower() not in person_award and word.lower() not in nomination_words]
#             tweet = ' '.join(tweet)
#             print(tweet)
#             if any([kw in tweet for kw in person_award]):
#                 person_list = ia.search_person(tweet)
#                 # Use bigram to extract name
#                 for p in person_list:
#                     if p[0]['name'] not in nominee[key]:
#                         nominee[key][p[0]['name']] = 1
#                     else:
#                         nominee[key][p[0]]['name'] += 1
#             else:
#                 #extracted_name = extract_from_tweet(tweet)
#                 #print(extracted_name)
#                 #if len(extracted_name) != 0:
#                 #print(key)
#                 movies = ia.search_movie(tweet)
#                 movies = [movie for movie in movies if 'year' in movie.keys() and movie['year'] <= 2013]
#                 for i in range(len(movies)):
#                     if movies[i]['title'] not in nominee:
#                         nominee[key][movies[i]['title']] = 1
#                     else:
#                         nominee[key][movies[i]['title']] += 1

nominee = {}
ia = IMDb()
for key, value in tweet_by_award_dict.items():
    nominee[key] = {}
    if any([kw in key for kw in person_award]):
        for tweet in value:
            if any([kw in tweet for kw in person_award]):   
                #print(tweet)
                tweet = tweet.split()
                tweet = [word for word in tweet if word.lower() not in person_award and word.lower() not in nomination_words]
                tweet = ' '.join(tweet)
                #print(key)
                #print(tweet)
                names = re.findall(name_pattern, tweet)
                #print(names)
                for name in names:
                    if name not in nominee[key]:
                        nominee[key][name] = 1
                    else:
                        nominee[key][name] += 1
    else:
        winner = ["musical","comedy","motion", "picture","golden","globe","movie","television","best","or","tv","original","series","animated",
                  "feature","film","song","drama","-","rt","to","goes","foreign",'the']
        for tweet in value:
            if not any([kw in tweet for kw in person_award]):
                tweet = tweet.split()
                tweet = [word for word in tweet if word.lower() not in winner and word.lower() not in nomination_words]
                tweet = ' '.join(tweet)
                names = re.findall(name_pattern, tweet)
                names = ' '.join(names)
                if names != '':
                    print(names)
                    # for name in names:
                    movies = ia.search_movie(names)
                    if len(movies) != 0:
                        for m in movies:
                            #print(m)
                            if "year" in m:
                                if m["year"] == 2013 or m["year"] == 2012 or m["year"] == 2011:
                                    if m["title"] not in nominee[key]:
                                        nominee[key][m["title"]] = 1
                                    else:
                                        nominee[key][m["title"]] += 1

Ben Affleck
Argo Ben
Argo Excellent
Lincoln But
Oh Argo
President Clinton
Ben Afleck
Ben Affleck
Affleck Mr Affleck Director
Argo Ben
Director Love
No Aaron Paul Peter
John Goodman
Werner Herzog Christoph Waltz
Lincoln Silver Linings Playbooks


KeyboardInterrupt: 

In [ ]:
for key, value in nominee.items():
    value_2 = nltk.FreqDist(value).most_common(5)
    nominee[key] = [n[0] for n in value_2]

In [ ]:

# def extract_noun_from_tweet(tweet):
#     # Tokenize and tag the tweet
#     tokens = word_tokenize(tweet)
#     tagged_tokens = pos_tag(tokens)
    
#     extracted_nouns = []
    
#     for i, (word, tag) in enumerate(tagged_tokens):
#         # If the word is one of the verbs and the next word is in nominated words, 
#         # extract the noun before the verb
#         if word in verbs and i < len(tagged_tokens) - 1 and tagged_tokens[i + 1][0] in nomination_words:
#             if i > 0 and 'NN' in tagged_tokens[i - 1][1]:  # Check if it's a noun
#                 extracted_nouns.append(tagged_tokens[i - 1][0])
        
#         # If the word is in nominated words and there's no verb before it,
#         # extract the noun after it
#         elif word in nomination_words and (i == 0 or tagged_tokens[i - 1][0] not in verbs):
#             if i < len(tagged_tokens) - 1 and 'NN' in tagged_tokens[i + 1][1]:  # Check if it's a noun
#                 extracted_nouns.append(tagged_tokens[i + 1][0])

#     return extracted_nouns
def extract_from_tweet(tweet):
    # Tokenize and tag the tweet
    tokens = word_tokenize(tweet)
    
    for i, word in enumerate(tokens):
        # If the word is one of the verbs and the next word is in nominated words, 
        # extract everything before the verb
        if word in verbs and i < len(tokens) - 1 and tokens[i + 1] in nomination_words:
            return ' '.join(tokens[:i])
        
        # If the word is in nominated words and there's no verb before it,
        # extract everything after it
        elif word in nomination_words and (i == 0 or tokens[i - 1] not in verbs):
            return ' '.join(tokens[i+1:])

    return []  # Return None if no matching pattern is found

In [ ]:
from nameparser import HumanName
from nltk import bigrams
from nltk.tokenize import word_tokenize

In [ ]:
test = {}
ia = IMDb()
for key, value in nominee_names.items():
    if key not in test:
        test[key] = {}
    for tweet in value:
        if any([kw in key for kw in person_award]):
            bigrams = nltk.bigrams(tweet.split())
            # use bigram list to extract name
            
            
            # names = extract_name(tweet)
            # for name in names:
            #     if name not in test[key]:
            #         test[key][name] = 1
            #     else:
            #         test[key][name] += 1
            name = HumanName(tweet)
            if name.full_name not in test[key]:
                test[key][name.full_name] = 1
            else:
                test[key][name.full_name] += 1
        else:
            extracted_name = extract_from_tweet(tweet)
            #print(extracted_name)
            if len(extracted_name) != 0:
                movies = ia.search_movie(''.join(extracted_name))
                movies = [movie for movie in movies if 'year' in movie.keys() and movie['year'] <= 2013]
                for i in range(len(movies)):
                    if movies[i]['title'] not in test:
                        test[key][movies[i]['title']] = 1
                    else:
                        test[key][movies[i]['title']] += 1

In [ ]:
for key, value in test.items():
    test[key] = nltk.FreqDist(value)
    print(key, test[key].most_common(5))

In [ ]:
ia = IMDb()

In [ ]:
p = ia.search_person("supporting actress")

In [ ]:
p[0]['name']

In [ ]:
text

In [ ]:
movie = ia.search_movie("Zero Dark")

In [ ]:
movie

In [ ]:
movie[0]["year"]

In [ ]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

In [ ]:
text = "Jessica Chastain Zero Dark"

In [ ]:
nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
for nltk_result in nltk_results:
    if type(nltk_result) == Tree:
        name = ''
        for nltk_result_leaf in nltk_result.leaves():
            name += nltk_result_leaf[0] + ' '
            print(name)
        print ('Type: ', nltk_result.label(), 'Name: ', name)